# Лабораторная работа №3 (Проведение исследований с решающим деревом)

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    mean_squared_error, mean_absolute_error, r2_score
)


# Создание бейзлайна и оценка качества 
**2.a** Обучить модели из `sklearn` (для классификации и регрессии) для выбранных наборов данных  
**2.b** Оценить качество моделей (для классификации и регрессии) по выбранным метрикам на выбранных наборах данных

In [23]:
# 1. Загрузка данных
data = pd.read_csv('student_data.csv', sep=',')

# 2. Классификация: задача "сдал/не сдал"
data_class = data.copy()
data_class['passed'] = (data_class['G3'] >= 10).astype(int)
X_class = data_class[['G1', 'G2', 'studytime', 'failures', 'absences']]
y_class = data_class['passed']

Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Бейзлайновая модель: DecisionTreeClassifier без особых настроек
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(Xc_train, yc_train)
yc_pred = dt_clf.predict(Xc_test)

acc = accuracy_score(yc_test, yc_pred)
prec = precision_score(yc_test, yc_pred)
rec = recall_score(yc_test, yc_pred)
f1 = f1_score(yc_test, yc_pred)

print("Бейзлайн DecisionTree (Classification):")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-score:  {f1:.4f}")

# 3. Регрессия: предсказание оценки G3
X_reg = data[['G1', 'G2', 'studytime', 'failures', 'absences']]
y_reg = data['G3']

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Бейзлайновая модель: DecisionTreeRegressor без особых настроек
dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(Xr_train, yr_train)
yr_pred = dt_reg.predict(Xr_test)

mse = mean_squared_error(yr_test, yr_pred)
mae = mean_absolute_error(yr_test, yr_pred)
r2 = r2_score(yr_test, yr_pred)

print("\nБейзлайн DecisionTree (Regression):")
print(f"MSE:  {mse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"R^2:  {r2:.4f}")


Бейзлайн DecisionTree (Classification):
Accuracy:  0.8734
Precision: 0.9574
Recall:    0.8491
F1-score:  0.9000

Бейзлайн DecisionTree (Regression):
MSE:  2.2500
MAE:  1.0443
R^2:  0.8903


# 3. Улучшение бейзлайна
**3.a** Сформулировать гипотезы (препроцессинг, визуализация, новые признаки, подбор гиперпараметров и т.д.)  
**3.b** Проверить гипотезы  
**3.c** Сформировать улучшенный бейзлайн  
**3.d** Обучить модели с улучшенным бейзлайном (для классификации и регрессии)  
**3.e** Оценить качество моделей  
**3.f** Сравнить результаты с пунктом 2  
**3.g** Сделать выводы

In [24]:
# Пример гипотезы улучшения:
# 1) Подбор гиперпараметров решающего дерева (max_depth, min_samples_split, min_samples_leaf...)
# 2) Масштабирование признаков, если считаем нужным (хотя дерево не так критично реагирует)
# 3) Добавление новых признаков (например, total_G1_G2 = G1 + G2)
# 4) Удаление выбросов (например, absences > 40)

# Для демонстрации: создадим признак total_G1_G2
data_class['total_G1_G2'] = data_class['G1'] + data_class['G2']
Xc_train2 = data_class.loc[Xc_train.index, ['G1','G2','studytime','failures','absences','total_G1_G2']]
Xc_test2  = data_class.loc[Xc_test.index,  ['G1','G2','studytime','failures','absences','total_G1_G2']]

# 3.1 Улучшение для классификации
param_grid_clf = {
    'max_depth': [3, 5, 7, None],
    'min_samples_leaf': [1, 2, 5, 10],
    'criterion': ['gini','entropy']
}
clf_cv = GridSearchCV(DecisionTreeClassifier(random_state=42),
                      param_grid_clf, cv=5, scoring='f1', n_jobs=-1)
clf_cv.fit(Xc_train2, yc_train)

best_clf = clf_cv.best_estimator_
yc_pred_best = best_clf.predict(Xc_test2)

acc_best = accuracy_score(yc_test, yc_pred_best)
prec_best = precision_score(yc_test, yc_pred_best)
rec_best = recall_score(yc_test, yc_pred_best)
f1_best = f1_score(yc_test, yc_pred_best)

print("Улучшенный бейзлайн (DecisionTree Classifier):")
print("Лучшие параметры:", clf_cv.best_params_)
print(f"Accuracy:  {acc_best:.4f}")
print(f"Precision: {prec_best:.4f}")
print(f"Recall:    {rec_best:.4f}")
print(f"F1-score:  {f1_best:.4f}")

data.dropna(inplace=True)
print("Shape after dropna:", data.shape)
data = data[data['absences'] <= 40]
categorical_cols = ['Mjob', 'Fjob', 'reason', 'guardian']
binary_cols = ['schoolsup', 'famsup', 'paid', 'activities', 'nursery', 
               'higher', 'internet', 'romantic']

# Преобразуем yes/no -> 1/0
for col in binary_cols:
    data[col] = data[col].map({'yes':1, 'no':0})
data['total_G1_G2'] = data['G1'] + data['G2']

numeric_cols = ['G1', 'G2', 'studytime', 'failures', 'absences', 'total_G1_G2'] + binary_cols

X = data[numeric_cols + categorical_cols].copy()
y = data['G3'].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")


numeric_transformer = StandardScaler()  # Можно и не масштабировать — иногда не нужно для деревьев
cat_transformer = OneHotEncoder(drop='first', handle_unknown='ignore')

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_cols),
    ("cat", cat_transformer, categorical_cols)
])

# 7.2 Пайплайн: сперва preprocessor -> потом DecisionTreeRegressor
pipe_dt = Pipeline([
    ("preprocess", preprocessor),
    ("dt", DecisionTreeRegressor(random_state=42))
])

# =========================
# 8. ПОДБОР ГИПЕРПАРАМЕТРОВ
# =========================
param_grid = {
    "dt__max_depth": [3, 5, 7, 10, None],
    "dt__min_samples_leaf": [1, 2, 5, 10],
    "dt__min_samples_split": [2, 5, 10],
    "dt__criterion": ["squared_error", "absolute_error"]
    # Можно добавить/убрать любые параметры
}

grid_search = GridSearchCV(
    pipe_dt,
    param_grid,
    cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
print("Лучшие параметры:", grid_search.best_params_)

# =========================
# 9. ОЦЕНКА КАЧЕСТВА УЛУЧШЕННОГО БЕЙЗЛАЙНА
# =========================
best_dt = grid_search.best_estimator_
y_pred = best_dt.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n=== Улучшенный бейзлайн DecisionTree (Regression) ===")
print(f"MSE:  {mse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"R^2:  {r2:.4f}")


Улучшенный бейзлайн (DecisionTree Classifier):
Лучшие параметры: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 10}
Accuracy:  0.8861
Precision: 0.9583
Recall:    0.8679
F1-score:  0.9109
Shape after dropna: (395, 33)
Train size: (313, 18), Test size: (79, 18)
Лучшие параметры: {'dt__criterion': 'squared_error', 'dt__max_depth': 3, 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 2}

=== Улучшенный бейзлайн DecisionTree (Regression) ===
MSE:  3.1542
MAE:  1.1418
R^2:  0.7744


# 4. Имплементация алгоритма машинного обучения (Random Forest)

**4.a** Самостоятельно имплементировать алгоритмы (для классификации и регрессии)  
**4.b** Обучить имплементированные модели (для классификации и регрессии)  
**4.c** Оценить качество имплементированных моделей  
**4.d** Сравнить результаты имплементированных моделей с пунктом 2  
**4.e** Сделать выводы  
**4.f** Добавить техники из улучшенного бейзлайна (пункт 3.c)  
**4.g** Обучить модели (для классификации и регрессии)  
**4.h** Оценить качество моделей  
**4.i** Сравнить результаты моделей с пунктом 3  
**4.j** Сделать выводы

In [26]:
# Ячейка [4] (Code)

import numpy as np
from collections import Counter

########################################################################
# Пример упрощенной имплементации дерева "с нуля" (для демонстрации)
# В реальной практике это намного более объёмный код 
# и обычно используют готовые библиотеки.
########################################################################

class CustomDecisionTreeClassifier:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, X, y):
        X, y = np.array(X), np.array(y)
        self.root = self._build_tree(X, y, depth=0)
        return self

    def _build_tree(self, X, y, depth):
        # Простейшая рекурсия; неточная реализация
        # 1) критерий остановки
        if len(set(y)) == 1:
            return {'type':'leaf','class':y[0]}
        if self.max_depth is not None and depth >= self.max_depth:
            # Мажоритарный класс
            return {'type':'leaf','class': Counter(y).most_common(1)[0][0]}
        if len(X) < self.min_samples_split:
            return {'type':'leaf','class': Counter(y).most_common(1)[0][0]}

        # 2) Выбираем лучший сплит (очень упрощённый)
        best_feat = 0
        best_thresh = np.median(X[:,0])
        # Можно реализовать поиск по всем признакам и 
        # разным значениям, считаем прирост информации, Gini, и т.д.

        # 3) Делим выборку
        left_idx = X[:,best_feat] <= best_thresh
        right_idx = ~left_idx

        # 4) Построение узлов
        node = {
            'type': 'node',
            'feature': best_feat,
            'thresh': best_thresh,
            'left':  self._build_tree(X[left_idx], y[left_idx], depth+1),
            'right': self._build_tree(X[right_idx], y[right_idx], depth+1)
        }
        return node

    def predict(self, X):
        X = np.array(X)
        preds = []
        for x in X:
            preds.append(self._traverse(self.root, x))
        return np.array(preds)

    def _traverse(self, node, x):
        if node['type'] == 'leaf':
            return node['class']
        else:
            if x[node['feature']] <= node['thresh']:
                return self._traverse(node['left'], x)
            else:
                return self._traverse(node['right'], x)


class CustomDecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, X, y):
        X, y = np.array(X), np.array(y)
        self.root = self._build_tree(X, y, depth=0)
        return self

    def _build_tree(self, X, y, depth):
        if self.max_depth is not None and depth >= self.max_depth:
            return {'type':'leaf', 'value': y.mean()}
        if len(X) < self.min_samples_split:
            return {'type':'leaf', 'value': y.mean()}
        if np.allclose(y, y[0]):
            return {'type':'leaf', 'value': y[0]}

        # Упрощённый сплит - берем первый признак
        best_feat = 0
        best_thresh = np.median(X[:,0])

        left_idx = X[:,best_feat] <= best_thresh
        right_idx = ~left_idx

        node = {
            'type': 'node',
            'feature': best_feat,
            'thresh': best_thresh,
            'left': self._build_tree(X[left_idx], y[left_idx], depth+1),
            'right': self._build_tree(X[right_idx], y[right_idx], depth+1)
        }
        return node

    def predict(self, X):
        X = np.array(X)
        preds = []
        for x in X:
            preds.append(self._traverse(self.root, x))
        return np.array(preds)

    def _traverse(self, node, x):
        if node['type'] == 'leaf':
            return node['value']
        else:
            if x[node['feature']] <= node['thresh']:
                return self._traverse(node['left'], x)
            else:
                return self._traverse(node['right'], x)


# 4.a-b Обучим кастомные модели
# Для классификации
cust_clf = CustomDecisionTreeClassifier(max_depth=3, min_samples_split=5)
cust_clf.fit(Xc_train, yc_train)
yc_pred_custom = cust_clf.predict(Xc_test)

acc_cust = accuracy_score(yc_test, yc_pred_custom)
f1_cust = f1_score(yc_test, yc_pred_custom)

print("Custom Decision Tree - Classification:")
print(f"Accuracy: {acc_cust:.4f}")
print(f"F1-score: {f1_cust:.4f}")

# Для регрессии
cust_reg = CustomDecisionTreeRegressor(max_depth=3, min_samples_split=5)
cust_reg.fit(Xr_train, yr_train)
yr_pred_custom = cust_reg.predict(Xr_test)

mse_cust = mean_squared_error(yr_test, yr_pred_custom)
r2_cust = r2_score(yr_test, yr_pred_custom)

print("\nCustom Decision Tree - Regression:")
print(f"MSE: {mse_cust:.4f}")
print(f"R^2: {r2_cust:.4f}")

# 4.c - Оценить качество (см. выше)
# 4.d - Сравнить с пунктом 2 (было dt_clf, dt_reg из sklearn)
# 4.e - Выводы

# 4.f-g - Применить улучшения (например, меняем параметры, добавляем новые фичи)
# ...


Custom Decision Tree - Classification:
Accuracy: 0.7848
F1-score: 0.8247

Custom Decision Tree - Regression:
MSE: 6.1922
R^2: 0.6980
